# Plot different types of mutations versus time

In [1]:
import datetime
import json
import urllib

import altair as alt

import pandas as pd

Dates taken from Richard Neher's repo [Snakefile](https://github.com/neherlab/SC2_variant_rates/blob/master/Snakefile):

In [2]:
dates = {
    '19A':  2019.9,
    '19B':  2019.9,
    #'19B+': 2019.9,
    #'19B++': 2019.9,
    '20A':  2020.1,
    '20B':  2020.1,
    '20C':  2020.1,
    #'20A+': 2020.1,
    '20E':  2020.47,
    '20H':  2020.63,
    '20I':  2020.715,
    '20J':  2020.91,
    '21D':  2020.9,
    '21G':  2021.0,
    '21H':  2021.0,
    '21I':  2021.22,
    '21J':  2021.21,
    '21K':  2021.85,
    '21L':  2021.85,
    '22A':  2022.163,
    '22B':  2022.163,
    '22D':  2022.2,
    #'21L+': 2021.8,
}

Read clade mutation information also from Richard Neher's repo:

In [3]:
muts_json = "https://raw.githubusercontent.com/neherlab/SC2_variant_rates/master/data/clade_gts.json"

with urllib.request.urlopen(muts_json) as f:
    muts = json.load(f)

Now build data frame:

In [4]:
spike_start, spike_end = (21563, 25381)  # start and stop of spike

records = []
for clade, date in dates.items():
    clade_muts = muts[clade]
    n_total = len(clade_muts["nuc"])
    n_nonsyn = len(clade_muts["aa"])
    n_syn = n_total - n_nonsyn
    n_spike_nonsyn = sum(m.startswith("S:") for m in clade_muts["aa"])
    records.append(
        (
            clade,
            date,
            n_total,
            n_nonsyn - n_spike_nonsyn,
            n_syn,
            n_spike_nonsyn,
        )
    )
    
df = pd.DataFrame(
    records,
    columns=[
        "clade",
        "date",
        "nucleotide mutations",
        "non-spike amino-acid mutations",
        "synonymous mutations",
        "spike amino-acid mutations",
    ]
).melt(
    id_vars=["clade", "date"],
    var_name="mutation_type",
    value_name="number of mutations",
).assign(
    is_Omicron=lambda x: x["clade"].isin(["21K", "21L", "22A", "22B", "22C", "22D"]),
    is_Omicron_str=lambda x: x["is_Omicron"].map({True: "Omicron clade", False: "pre-Omicron clade"}),
    mutation_type=lambda x: x["mutation_type"].str.replace(" mutations", ""),
)

# convert date to datetime https://stackoverflow.com/a/20911144
def year_to_date(decimal_year):
    year = int(decimal_year)
    rem = decimal_year - year
    base = datetime.datetime(year, 1, 1)
    return base + datetime.timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)

#df["date"] = df["date"].map(year_to_date)

Now plot the data:

In [5]:
scalefac = 2.5

mut_type_selection = alt.selection_single(
    fields=["mutation_type"],
    bind=alt.binding_select(
        name="mutation type",
        options=df["mutation_type"].unique().tolist(),
    ),
    init=[{"mutation_type": "nucleotide"}],
)

omicron_selection = alt.selection_multi(
    fields=['is_Omicron_str'],
    bind='legend',
    toggle="true",
    init=[{"is_Omicron_str": "pre-Omicron clade"}],
)

base = alt.Chart(df).transform_filter(mut_type_selection)

chart = (
    base
    .encode(
        x=alt.X(
            "date",
            scale=alt.Scale(zero=False, nice=False),
            axis=alt.Axis(format=".1f", values=[2020, 2021, 2022]),
        ),
        y=alt.Y("number of mutations", scale=alt.Scale(nice=False)),
        color=alt.Color(
            "is_Omicron_str",
            scale=alt.Scale(domain=["pre-Omicron clade", "Omicron clade"]),
        ),
        shape=alt.Shape(
            "is_Omicron_str",
            title="click to select/de-select",
            legend=alt.Legend(orient="right"),
            scale=alt.Scale(domain=["pre-Omicron clade", "Omicron clade"]),
        ),
        opacity=alt.condition(omicron_selection, alt.value(1), alt.value(0)),
        tooltip=["clade", "date", "number of mutations"],
    )
    .mark_point(filled=True, size=60 * scalefac)
)

trendline = (
    base
    .encode(x="date",y="number of mutations")
    .transform_filter(~alt.datum.is_Omicron)
    .transform_regression(
        "date",
        "number of mutations",
        extent=(df["date"].min() - 0.1, df["date"].max() + 0.1),
    )
    .mark_line(color="gray", size=5, opacity=0.4)
)

chart_w_line = (
    (chart + trendline)
    .properties(width=275 * scalefac, height=165 * scalefac)
    .configure_axis(grid=False, labelFontSize=11 * scalefac, titleFontSize=12 * scalefac)
    .configure_legend(labelFontSize=12 * scalefac, titleFontSize=12 * scalefac, titleLimit=10000, labelLimit=10000)
    .add_selection(mut_type_selection)
    .add_selection(omicron_selection)
)

chart_w_line.save("chart.html")

chart_w_line

alt.LayerChart(...)